# Data Mining Analysis on Breast Cancer Project

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization library  
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import time
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv('../input/data.csv')

In [ ]:
data.head()  # head method show only first 5 rows

In [ ]:
# feature names as a list
col = data.columns       # .columns gives columns names in data 
print(col)

In [ ]:
# y includes our labels and x includes our features
y = data.diagnosis                          # M or B 
list = ['Unnamed: 32','id','diagnosis']
x = data.drop(list,axis = 1 )   # drop useless features
x.head()

In [ ]:
ax = sns.countplot(y,label="Count")# M = 212, B = 357
ax.grid(False)
ax.set_xticklabels(["Malignant","Benign"])
B, M= y.value_counts()
print('Number of Benign: ',B)
print('Number of Malignant : ',M)
print("M/B percentage: ",M/B)  # to check the possible selection bias for future model building

In [ ]:
x.describe()

# Visualization


In [ ]:
# first ten features
data_dia = y
data = x
data_n_2 = (data - data.mean()) / (data.std())              # standardization
data = pd.concat([y,data_n_2.iloc[:,0:10]],axis=1)
data = pd.melt(data,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(10,10))
sns.violinplot(x="features", y="value", hue="diagnosis", data=data,split=True, inner="quart")
plt.xticks(rotation=90)

In [ ]:
# Second ten features
data = pd.concat([y,data_n_2.iloc[:,10:20]],axis=1)
data = pd.melt(data,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(10,10))
sns.violinplot(x="features", y="value", hue="diagnosis", data=data,split=True, inner="quart")
plt.xticks(rotation=90)

In [ ]:
# Third ten features
data = pd.concat([y,data_n_2.iloc[:,20:31]],axis=1)
data = pd.melt(data,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(10,10))
sns.violinplot(x="features", y="value", hue="diagnosis", data=data,split=True, inner="quart")
plt.xticks(rotation=90)

In [ ]:
#use boxplot to check  similarity
plt.figure(figsize=(10,10))
sns.boxplot(x="features", y="value", hue="diagnosis", data=data)
plt.xticks(rotation=90)

In [ ]:
sns.jointplot(x.loc[:,'concavity_worst'], x.loc[:,'concave points_worst'], kind="regg", color="#ce1414")

In [ ]:
sns.set(style="white")
df = x.loc[:,['radius_worst','perimeter_worst','area_worst']]
g = sns.PairGrid(df, diag_sharey=False)
g.map_lower(sns.kdeplot, cmap="Blues_d")
g.map_upper(plt.scatter)
g.map_diag(sns.kdeplot, lw=3)

In [ ]:
sns.set(style="whitegrid", palette="muted")
data_dia = y
data = x
data_n_2 = (data - data.mean()) / (data.std())              # standardization
data = pd.concat([y,data_n_2.iloc[:,0:10]],axis=1)
data = pd.melt(data,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(10,10))
tic = time.time()
sns.swarmplot(x="features", y="value", hue="diagnosis", data=data)

plt.xticks(rotation=90)

In [ ]:
data = pd.concat([y,data_n_2.iloc[:,10:20]],axis=1)
data = pd.melt(data,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(10,10))
sns.swarmplot(x="features", y="value", hue="diagnosis", data=data)
plt.xticks(rotation=90)

In [ ]:
data = pd.concat([y,data_n_2.iloc[:,20:31]],axis=1)
data = pd.melt(data,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(10,10))
sns.swarmplot(x="features", y="value", hue="diagnosis", data=data)
toc = time.time()
plt.xticks(rotation=90)
print("swarm plot time: ", toc-tic ," s")

In [ ]:
#correlation map
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap(x.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

# Feature Selection and Random Forest Classification


In this part we will select feature with different methods that are feature selection with correlation, univariate feature selection, recursive feature elimination (RFE), recursive feature elimination with cross validation (RFECV) and tree based feature selection. We will use random forest classification in order to train our model and predict. 

## 1) Feature selection with correlation and random forest classification

In [ ]:
drop_list1 = ['perimeter_mean','radius_mean','compactness_mean','concave points_mean','radius_se','perimeter_se','radius_worst','perimeter_worst','compactness_worst','concave points_worst','compactness_se','concave points_se','texture_worst','area_worst']
x_1 = x.drop(drop_list1,axis = 1 )        # do not modify x, we will use it later 
x_1.head()

    

In [ ]:
#correlation map
f,ax = plt.subplots(figsize=(14, 14))
sns.heatmap(x_1.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score,confusion_matrix
from sklearn.metrics import accuracy_score

# split data train 80 % and test 20 %
x_train, x_test, y_train, y_test = train_test_split(x_1, y, test_size=0.2, random_state=42)

#random forest classifier with n_estimators=10 (default)
clf_rf = RandomForestClassifier(random_state=43)      
clr_rf = clf_rf.fit(x_train,y_train)

ac = accuracy_score(y_test,clf_rf.predict(x_test))
print('Accuracy is: ',ac)
cm = confusion_matrix(y_test,clf_rf.predict(x_test))
sns.heatmap(cm,annot=True,fmt="d")

## 2) Univariate feature selection and random forest classification
In univariate feature selection, we will use SelectKBest that removes all but the k highest scoring features.
<http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest>
we consider choose top 5 features

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# find best scored 5 features
select_feature = SelectKBest(chi2, k=5).fit(x_train, y_train)
feature_importance={}

In [ ]:
print('Score list:', select_feature.scores_)
print('Feature list:', x_train.columns)

In [ ]:
x_train_2 = select_feature.transform(x_train)
x_test_2 = select_feature.transform(x_test)
#random forest classifier with n_estimators=10 (default)
clf_rf_2 = RandomForestClassifier()      
clr_rf_2 = clf_rf_2.fit(x_train_2,y_train)
ac_2 = accuracy_score(y_test,clf_rf_2.predict(x_test_2))
print('Accuracy is: ',ac_2)
cm_2 = confusion_matrix(y_test,clf_rf_2.predict(x_test_2))
sns.heatmap(cm_2,annot=True,fmt="d")

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(random_state=0, solver='lbfgs',class_weight=True)
scores = cross_val_score(clf, x_train_2,y_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
def model_scores(model,x_train_2,y_train):
    clf=model
    scores = cross_val_score(clf, x_train_2,y_train, cv=10)
    return scores.mean()

### Here we try several classification algorithm like Logistic Regression, Naive Bayes, SVM Linear, QDA, KNN to see different model's performance on difference feature space(5 features and 15 features) 

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
model_list=[LogisticRegression(random_state=0, solver='lbfgs',class_weight=True),GaussianNB(priors=[0.6,0.4]),SVC(class_weight="balanced",kernel="linear"),QuadraticDiscriminantAnalysis(priors=[0.6,0.4]),KNeighborsClassifier(n_neighbors=3)]
model_name=["Logistic Regression","Naive Bayes","SVM Linear","QDA","KNN"]
performance_of_5_features_of_train={}
print("Main 5 features scores with different models in train")
for i,name in enumerate(model_name):
    performance_of_5_features_of_train[name]=model_scores(model_list[i],x_train_2,y_train)
print(performance_of_5_features_of_train)
print("All features scores with different models in train")    
performance_of_15_features_of_train={}
for i,name in enumerate(model_name):
    performance_of_15_features_of_train[name]=model_scores(model_list[i],x_train,y_train)
print(performance_of_15_features_of_train)

In [ ]:
%pylab inline
fig=figure(figsize=(20,10))
ax1=subplot(121)
ax1=scatter(performance_of_5_features_of_train.keys(),performance_of_5_features_of_train.values())
ylim(0.8,1.0)
ax2=subplot(122)
ax2=scatter(performance_of_15_features_of_train.keys(),performance_of_15_features_of_train.values())
ylim(0.8,1.0)

In [ ]:
print("Main 5 features scores with different models in test")
performance_of_5_features_in_test={}
performance_of_15_features_in_test={}
for i,name in enumerate(model_name):
    performance_of_5_features_in_test[name]=model_scores(model_list[i],x_test_2,y_test)
print(performance_of_5_features_in_test)
print("All features scores with different models in test")    
for i,name in enumerate(model_name):
    performance_of_15_features_in_test[name]=model_scores(model_list[i],x_test,y_test)
print(performance_of_15_features_in_test)

In [ ]:
fig=figure(figsize=(20,10))
ax1=subplot(121)
ax1=scatter(performance_of_5_features_in_test.keys(),performance_of_5_features_in_test.values())
ylim(0.8,1.0)
ax2=subplot(122)
ax2=scatter(performance_of_15_features_in_test.keys(),performance_of_15_features_in_test.values())
ylim(0.8,1.0)

###  We try to see our model's performances on the train and test based on different feature space. The following is 15 features

In [ ]:
fig=figure(figsize=(20,10))
grid(False)
scatter(performance_of_15_features_of_train.keys(),performance_of_15_features_of_train.values(),color="r",s=100)
scatter(performance_of_15_features_in_test.keys(),performance_of_15_features_in_test.values(),color="b",s=100)
ylim(0.8,1.0) 
legend(["train","test"],fontsize=20)

### Here we try to see whether our 5 features perform well on the models 

In [ ]:
fig=figure(figsize=(20,10))
scatter(performance_of_5_features_of_train.keys(),performance_of_5_features_of_train.values(),c="r")
scatter(performance_of_5_features_of_train.keys(),performance_of_5_features_in_test.values(),c="b")
ylim(0.8,1.0)

### Our findings is interesting:  apparently some classification algorithm has overfitting during crossvalidation when we use all features to predict. However, when we try to use just five main features model to test our algorithm performance, almost all models have better performances. This implies if we use all features, we have overfitting problem. At the same time, the main five features could better indicate or explain whether someone would have cancer.   What we can learn from these is another interesting phenomenon since QDA has better performance if we use all features instead of 5 features whenever we are training or testing.  However, QDA has weaker interpretability as logistic regression. 

## Our conclusion: 1. only 5 main features really matter.  2. Logistic regression is a pretty good model to do classification in this case in terms of performance and interpretability. 
                                                   

## 3) Recursive feature elimination (RFE) with random forest


In [ ]:
from sklearn.feature_selection import RFE
# Create the RFE object and rank each pixel
clf_rf_3 = RandomForestClassifier()      
rfe = RFE(estimator=clf_rf_3, n_features_to_select=5, step=1)
rfe = rfe.fit(x_train, y_train)


In [ ]:
print('Chosen best 5 feature by rfe:',x_train.columns[rfe.support_])

Chosen 5 best features by rfe is **texture_mean, area_mean, concavity_mean, area_se, concavity_worst**. They are exactly similar with previous (selectkBest) method. Therefore we do not need to calculate accuracy again. Shortly, we can say that we make good feature selection with rfe and selectkBest methods. However as you can see there is a problem, okey I except we find best 5 feature with two different method and these features are same but why it is **5**. Maybe if we use best 2 or best 15 feature we will have better accuracy. Therefore lets see how many feature we need to use with rfecv method.

## 4) Recursive feature elimination with cross validation and random forest classification
<http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html>
Now we will not only **find best features** but we also find **how many features do we need** for best accuracy.

In [ ]:
from sklearn.feature_selection import RFECV

# The "accuracy" scoring is proportional to the number of correct classifications
clf_rf_4 = RandomForestClassifier() 
rfecv = RFECV(estimator=clf_rf_4, step=1, cv=5,scoring='accuracy')   #5-fold cross-validation
rfecv = rfecv.fit(x_train, y_train)

print('Optimal number of features :', rfecv.n_features_)
print('Best features :', x_train.columns[rfecv.support_])

In [ ]:
# Plot number of features VS. cross-validation scores
import matplotlib.pyplot as plt
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score of number of selected features")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

## 5) Tree based feature selection and random forest classification
<http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html>
In random forest classification method there is a **feature_importances_** attributes that is the feature importances (the higher, the more important the feature). **!!! To use feature_importance method, in training data there should not be correlated features. Random forest choose randomly at each iteration, therefore sequence of feature importance list can change.**


In [ ]:
clf_rf_5 = RandomForestClassifier()      
clr_rf_5 = clf_rf_5.fit(x_train,y_train)
importances = clr_rf_5.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf_rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(x_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest

plt.figure(1, figsize=(14, 13))
plt.title("Feature importances")
plt.bar(range(x_train.shape[1]), importances[indices],
       color="g", yerr=std[indices], align="center")
plt.xticks(range(x_train.shape[1]), x_train.columns[indices],rotation=90)
plt.xlim([-1, x_train.shape[1]])
plt.show()

As you can seen in plot above, after 5 best features importance of features decrease. Therefore we can focus these 5 features. As I sad before, I give importance to understand features and find best of them. 